In [247]:
# Ataque de Coster al criptosistema de Merkle-Hellman
# Juan Manuel Mateos Pérez

In [248]:
## Explicación :
# En este programa estamos simulando el ataque mediante el método de Coster a una comunicación realizada usando el 
# criptosistema de Merkle-Hellman. El ataque utiliza únicamente los valores conocidos del criptosistema, que son la
# clave pública y el mensaje cifrado. El programa obtiene como resultado un mensaje descifrado, el cual comprobaremos
# si coincide con el original.

## Ejecución :
# Para ejecutar el programa, solo debemos descomentar el código del main y el import que queramos utilizar:
# (1) Si descomentamos la primera parte, podremos ejecutar el programa 1 vez y veremos todos los datos necesarios. Además, 
# podemos modificar el tamaño del mensaje (variable tam) y el número de iteraciones de la clave privada (variable it).
# (2) Si por otro lado descomentamos la segunda parte, el programa se ejecuta p veces y mostrará un desglose de fallos, 
# vacios y errores de longitud cometidos. En este caso podemos modificar la variable p, que indica la cantidad de cripto-
# sistemas que se van a ejecutar.
# (3) Finalmente, si descomentamos la tercera parte, el programa ejecuta una función que genera varios criptosistemas con
# con distintas densidades para así analizar el rendimiento del algoritmo. Aquí, se puede modificar el tamaño del mensaje 
# (variable tam), el número de ejecuciones totales (variable it) y el nombre del archivo donde debe mostrar los datos 
# (variable archivo).

In [249]:
# Importamos el método de Merkle-Hellman
# load("MH_Lagarias_Coster.py")                        # descomentar para ejecutar las dos primeras partes del Main
# load("MH_Grafica_Coster.py")                         # descomentar para ejecutar la tercera parte del Main

In [250]:
# ATAQUE DE COSTER

In [251]:
# Genera la matriz necesaria para aplicar Coster
def generarMatriz(pk, s):
    n = len(pk)
    N = random.randint(int((1/2)*sqrt(n)), int(sqrt(n)))
    filas = []
    
    # generamos los n primeros vectores
    for i in range(0, n):
        aux = zero_vector(n+1)
        aux[i] = 1
        aux[n] = pk[i]*N
        filas.append(aux)

    # generamos el vector n+1
    b = []
    for i in range(n+1):
        b.append(1/2)
    b[n] = s*N
        
    filas.append(b)
    
    return Matrix(filas)

In [252]:
# Encuentra una solución en la matriz recibida
def buscarSolucion(matriz, pk, s):
    n = matriz.nrows()
    sol_encontrada = False
    solucion = []
    
    for j in range(n):
        if sol_encontrada == False:
            suma = 0
            fila = matriz[j]
        
            for i in range(len(fila)-1):
                aux = fila[i] + (1/2)
                if aux < 0:
                    aux = 0
                suma += pk[i]*aux
                solucion.append(aux)
            if suma == s:
                sol_encontrada = True
            else:
                solucion = []

    solucion = list(solucion)
    return solucion

In [253]:
# Intercambia los 1/2 por -1/2 y viceversa en una matriz
def cambiarMatriz(matriz):
    matriz_res = copy(matriz)

    for i in range(matriz.nrows()):
        for j in range(matriz.ncols()-1):
            if matriz[i, j] == 1/2:
                matriz_res[i, j] = -1/2
            elif matriz[i, j] == -1/2:
                matriz_res[i, j] = 1/2
    
    return matriz_res

In [254]:
# Aplica el ataque de Coster
def ataqueCoster(pk, s):
    n = len(pk)
    encontrado = False
    solucion = []
    
    # generamos la matriz
    matriz_ini = generarMatriz(pk, s)
    # aplicamos LLL
    matriz_res = matriz_ini.LLL()
    # buscamos una solución
    solucion = buscarSolucion(matriz_res, pk, s)
    
    # cambiamos los 1/2 por -1/2 y viceversa
    if len(solucion) == 0:
        matriz_cambio = cambiarMatriz(matriz_res)
        solucion = buscarSolucion(matriz_cambio, pk, s)
        
    return solucion

In [255]:
# Calcula el número de errores cometidos
def comprobarErrores(men_orig, men_obt):
    n = len(men_orig)
    vector_dif = []

    if len(men_obt) == 0:
        return len(men_orig)

    for i in range(0, n):
        vector_dif.append(abs(men_orig[i] - men_obt[i]))

    return sum(vector_dif)

In [256]:
# DATOS DE SALIDA

In [257]:
# ejecuta y muestra los datos tras aplicar una iteración
def unaIteracion(tam, it):
    merkle_hellman = Merkle_Hellman(tam, it)
    merkle_hellman.do()

    pk = merkle_hellman.pk
    s  = merkle_hellman.s

    coster = ataqueCoster(pk, s)
    
    print()
    print("Clave pública      :", merkle_hellman.pk)
    print("Mensaje original   :", merkle_hellman.mensaje)
    print("Mensaje cifrado    :", merkle_hellman.s)
    print("Mensaje descifrado :", coster)
    print("Tamaño mensaje     :", tam)
    print("Número iteraciones :", it)
    print("Errores totales    :", comprobarErrores(merkle_hellman.mensaje, coster))

In [258]:
# ejecuta y muestra los datos tras aplicar n iteraciones
def variasIteraciones(n):
    errores  = 0
    err_long = 0
    vacios   = 0
    densidad = 0
    
    print()
    print("Iteración \t Tamaño vector \t Número Iteraciones \t Densidad \t\tResultado")
    
    for i in range(p):
        print(i+1, end = "")

        tam = random.randint(3, 100)
        print("\t\t", tam, end="")

        it = random.randint(0, 3)
        print("\t\t", it, end="")

        merkle_hellman = Merkle_Hellman(tam, it)
        merkle_hellman.do()

        densidad = RR(merkle_hellman.tamano / log(max(merkle_hellman.pk), 2))
        print("\t\t\t", densidad, end="")

        coster = ataqueCoster(merkle_hellman.pk, merkle_hellman.s)
        if len(coster) == 0:
            vacios += 1
            print("\tvacio")
        elif len(coster) != len(merkle_hellman.mensaje):
            err_long += 1
            print("\terror longitud")
        else:
            valor = comprobarErrores(merkle_hellman.mensaje, coster)
            errores += valor
            if valor != 0:
                print("\terror valor")
            else:
                print("\tobtenido")
    
    print()
    print("Errores totales  tras", p, "iteraciones :", errores)
    print("Vacios  totales  tras", p, "iteraciones :", vacios)
    print("Errores longitud tras", p, "iteraciones :", err_long)

In [259]:
# ejecuta criptosistemas con distintas densidades para analizar el rendimiento
def medirErrores(tam, num_it, archivo):
    inicio = tam * 5
    fin = tam // 2
    paso = (inicio - fin) / (num_it-1)
  
    for i in range(num_it):
        n = fin + i*paso
        lim_inf = int(2 ** (2*n + 1) + 1)
        lim_sup = int(2 ** (2*n + 2) - 1)
        m  = random.randint(lim_inf, lim_sup)
        
        obtenido = False
        for j in range(10):
            if obtenido == False:
                merkle_hellman = Merkle_Hellman(tam, m)
                merkle_hellman.do()
                densidad = RR(merkle_hellman.tamano / log(max(merkle_hellman.pk), 2))
                coster = ataqueCoster(merkle_hellman.pk, merkle_hellman.s)
                if coster == merkle_hellman.mensaje:
                    obtenido = True
        
        resultado = str(tam)
        resultado += "\t" + str(densidad)
        if obtenido == True:
           resultado += "\t 0"
        else:
            resultado += "\t 1"

        with open(archivo, 'a') as f:
            f.write(resultado + "\n")

In [260]:
# MAIN

In [261]:
print("Ataque de Coster")

# ---------- descomentar para realizar 1 ejecución aleatoria ----------
# tam = random.randint(3, 100)
# it  = random.randint(0, 3)
# unaIteracion(tam, it)

# ---------- descomentar para realizar p ejecuciones aleatorias ----------
# p = 5
# variasIteraciones(p)

# ---------- descomentar para analizar it ejecuciones aleatorias ----------
# tam = 10
# it = 50
# archivo = "resultados.txt"
# medirErrores(tam, it, archivo)
# print("Finalizado")

Ataque de Coster
